In [2]:
import pandas as pd
import requests

In [3]:
def extract_last_week_neo_data(start_date, end_date, api_key):
    """Получает JSON-данные по API"""
    try:
        with requests.Session() as session:
            response = session.get(f"https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key={api_key}")
            response.raise_for_status()
            data = response.json()
        
        return data
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе данных: {e}")
        return None

In [4]:
def get_last_week_neo(data):
    """Извлекаем список астероидов и конвертируем в DataFrame"""
    try:
        last_week_neo_data = []
        for date, objects in data["near_earth_objects"].items():
            for object in objects:
                last_week_neo_data.append(object)
        df = pd.json_normalize(last_week_neo_data)
        print("Данные об астероидах загружены")
        return df
    except Exception as e:
        print(f"Ошибка извлечения данных: {e}")
        return None

In [5]:
def get_close_approaches(data):
    """Извлекает данные о сближениях астероидов"""
    try:
        # Проверяем, что ключ 'close_approach_data' существует
        filtered_data = [
            obj for date, objects in data["near_earth_objects"].items()
            for obj in objects if 'close_approach_data' in obj
        ]
        # Нормализация данных о сближениях
        close_approach = pd.json_normalize(
            filtered_data, 
            'close_approach_data', 
            ['id', 'name'], 
            errors='ignore'  # Игнорируем отсутствующие поля
        )
        print("Данные о сближениях загружены")
        return close_approach
    except Exception as e:
        print(f"Ошибка при обработке данных о сближениях астероидов: {e}")
        return None


In [6]:
def get_asteroid_with_max_speed(close_approach):
    try:
        close_approach["relative_velocity.kilometers_per_second"] = close_approach["relative_velocity.kilometers_per_second"].astype(float)
        max_speed_index = close_approach["relative_velocity.kilometers_per_second"].idxmax()
        max_speed_df = close_approach.loc[max_speed_index, ['name', 'close_approach_date', 'relative_velocity.kilometers_per_second']]
        print("Данные об астероиде с максимальной скоростью получены")
        return max_speed_df
    except Exception as e:
        print(f"Ошибка при нахождении астероида с максимальной скоростью: {e}")
        return None

In [7]:
def write_last_week_data_to_excel(nasa_neo_last_week_df, close_approaches_df, asteroid_max_speed_df, filename):
    try:
        with pd.ExcelWriter(filename, engine="xlsxwriter") as writer:
            if nasa_neo_last_week_df is not None:
                nasa_neo_last_week_df.to_excel(writer, sheet_name="Asteroids", index=False)
            if close_approaches_df is not None:
                close_approaches_df.to_excel(writer, sheet_name="Close_Approaches", index=False)
            if asteroid_max_speed_df is not None:
                asteroid_max_speed_df.to_excel(writer, sheet_name="Asteroid_max_speed")
        print(f"Данные успешно сохранены в '{filename}'")
    except Exception as e:
        print(f"Ошибка записи данных в Excel: {e}")

In [8]:
def main(api_key):
    """Загружаем, обрабатываем и добавляем данные в DataFrame"""
    start_date = "2025-03-21"
    end_date = "2025-03-28"

    neo_data = extract_last_week_neo_data(start_date, end_date, api_key)
    nasa_neo_last_week_df = get_last_week_neo(neo_data)
    close_approaches_df = get_close_approaches(neo_data)
    asteroid_max_speed_df = get_asteroid_with_max_speed(close_approaches_df)
    write_last_week_data_to_excel(nasa_neo_last_week_df, close_approaches_df, asteroid_max_speed_df, 'files/last_week_neo_data.xlsx')
        

if __name__ == "__main__":
    API_KEY = "502SJWBgC67LUsXow8S2uPQtZC29d2iDQ7Og6Hkg"
    main(API_KEY)

Данные об астероидах загружены
Данные о сближениях загружены
Данные об астероиде с максимальной скоростью получены
Данные успешно сохранены в 'files/last_week_neo_data.xlsx'
